In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [1]:
import os
import pandas as pd
df = pd.read_csv('./data/COVID-19-DATA.csv')
task_names = list(df.columns[1:])
df["molregno"]=range(len(df["SMILES"]))
df.to_csv('./data/COVID-19-DATA_new.csv',index=False)

In [1]:
import os
import pandas as pd
df = pd.read_csv('./data/COVID-19-DATA.csv')
task_names_total = list(df.columns[1:])


In [2]:
os.getcwd()

'/home/zsq17/~visar_qy'

## hyperparameter screening
The hyperparam_screening would return the best parameter dictionary.
Users could also check the performances recorded in the log file and manually pick the best param.

In [3]:
task_names=task_names_total[1:5]
para_dict_DNN = {
    'model_name': 'VISAR_pytorch_demo',  # user specific
    'task_list': task_names,  # MUST BE A LIST!
    # input data related params:
    'dataset_file': './data/COVID-19-DATA_new.csv',
    'feature_type': 'Morgan',
    'id_field': 'molregno',
    'smiles_field': 'SMILES',  #
    'model_flag': 'MT',
    'add_features': None,
    'frac_train': 0.9,
    'rand_seed': 0,
    'batch_size': 100,
    'normalize': True,
    # model architecture related parameters:
    'layer_nodes': [256, 128, len(task_names)], #
    'dropouts': 0.3,
    # model training related parameters:
    'learning_rate': 0.001,
    'GPU': False,
    'epoch': 100, # training epoch of each round (saving model at the end of each round)
    'epoch_num': 5, # how many rounds
    'optimizer': 'Adam',
    # viz file processing related parameters:
    'model_architecture':'ST',
    'hidden_layer': 1
}

In [4]:
from visar.utils.pytorch_functions import hyperparam_screening
import os
import copy
from collections import OrderedDict
from visar.dataloader.pytorch_utils import compound_FP_loader
from visar.pytorch_regressor import pytorch_DNN_model

candidate_params_dict = OrderedDict([('layer_nodes', [[256,128,len(task_names)], [512,64,len(task_names)], 
                                                      [512,128,len(task_names)], [512,265,len(task_names)]]),
                                     ('dropouts', [0.2, 0.4]),
                                     ('learning_rate', [0.01, 0.001])])

/home/zsq17/anaconda3/envs/pych/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.bicluster module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)

Bad key "text.kerning_factor" on line 4 in
/home/zsq17/anaconda3/envs/pych/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [5]:
best_param = hyperparam_screening(pytorch_DNN_model, para_dict_DNN, candidate_params_dict, 
                     mode = 'grid_search', epoch = 10, epoch_num = 2)

Extracted dataset shape: (852, 6)


/home/zsq17/anaconda3/envs/pych/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


-----------------------------
{'model_name': 'VISAR_pytorch_demo', 'task_list': ['CHEMBL2363065', 'CHEMBL3707467', 'CHEMBL4581', 'CHEMBL5600'], 'dataset_file': '/home/zsq17/~visar_qy/data/COVID-19-DATA_new.csv', 'feature_type': 'Morgan', 'id_field': 'molregno', 'smiles_field': 'SMILES', 'model_flag': 'MT', 'add_features': None, 'frac_train': 0.9, 'rand_seed': 0, 'batch_size': 100, 'normalize': True, 'layer_nodes': [256, 128, 4], 'dropouts': 0.3, 'learning_rate': 0.001, 'GPU': False, 'epoch': 100, 'epoch_num': 5, 'optimizer': 'Adam', 'model_architecture': 'ST', 'hidden_layer': 1, 'mean_list': [-2.9004206167, -2.3558172474, -1.5568911996452022, -1.2787787346000001], 'std_list': [0.43299614209176923, 0.5196112859274986, 1.2277285256477377, 1.0470026182568197], 'mad_list': [0.3657161829600001, 0.4066321777199999, 1.002357703865224, 0.88236819802], 'ratio_list': [1.1839676838668303, 1.2778410425878648, 1.2248407139621453, 1.1865824500545838]}
Epoch: 1 Loss=15.776
======== Iteration 0 ======

Epoch: 1 Loss=21.216
======== Iteration 0 ======
Evaluating model
[0.9389389432327012, 0.5642970659110282, 0.8502834622359063, 0.6661722257579786]
[0.05612347, 0.39354205, 0.15133183, 0.35224453]
Training R2 score: 0.755
Training MSE score: 0.238
[0.7848436397951037, -3.045533102343148, 0.5850134915965901, 0.12727423872435917]
[0.17655201, 0.6858825, 0.3945253, 0.5060754]
Test R2 score: -0.387
Test MSE score: 0.441
Epoch: 1 Loss=3.452
======== Iteration 1 ======
Evaluating model
[0.9140868285567041, 0.8377212052384309, 0.9030913725616272, 0.7361027050037939]
[0.07896596, 0.14657584, 0.09795416, 0.2784561]
Training R2 score: 0.848
Training MSE score: 0.150
[0.457531886101299, 0.03825677528045246, 0.5941424339257434, -0.25403759672275705]
[0.44513595, 0.16305462, 0.3858464, 0.7271902]
Test R2 score: 0.209
Test MSE score: 0.430
[0.30782263635353313, -0.387047355155935, 0.6024788251311604, 0.3940776469642736]
[0.5679837, 0.23516098, 0.37792104, 0.3513617]
Current score: 0.308
Best score: 0

Epoch: 1 Loss=15.530
======== Iteration 0 ======
Evaluating model
[0.8802444323462282, 0.7115656910555208, 0.829483726253897, 0.6380364729551271]
[0.11007176, 0.2605239, 0.17235598, 0.3819325]
Training R2 score: 0.765
Training MSE score: 0.231
[0.7157304675611265, -0.8748763423645796, 0.6181077991465385, -0.1155943773905852]
[0.23326457, 0.31786785, 0.36306268, 0.64690983]
Test R2 score: 0.086
Test MSE score: 0.390
Epoch: 1 Loss=3.280
======== Iteration 1 ======
Evaluating model
[0.874904497517669, 0.8369984769242204, 0.9091327430293008, 0.7255993458494583]
[0.114979886, 0.14722864, 0.09184761, 0.28953892]
Training R2 score: 0.837
Training MSE score: 0.161
[0.738209397202144, -0.3412321015501123, 0.6059637975846759, 0.0678165170254601]
[0.21481892, 0.22739343, 0.37460792, 0.5405537]
Test R2 score: 0.268
Test MSE score: 0.339
[0.7379327204896938, -1.0730188586710616, 0.6113980011306204, -0.18954535140698536]
[0.21504596, 0.35146108, 0.3694417, 0.68979245]
Current score: 0.738
Best score

...

/home/zsq17/anaconda3/envs/pych/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


-----------------------------
{'model_name': 'VISAR_pytorch_demo', 'task_list': ['CHEMBL2363065', 'CHEMBL3707467', 'CHEMBL4581', 'CHEMBL5600'], 'dataset_file': '/home/zsq17/~visar_qy/data/COVID-19-DATA_new.csv', 'feature_type': 'Morgan', 'id_field': 'molregno', 'smiles_field': 'SMILES', 'model_flag': 'MT', 'add_features': None, 'frac_train': 0.9, 'rand_seed': 2, 'batch_size': 100, 'normalize': True, 'layer_nodes': [256, 128, 4], 'dropouts': 0.3, 'learning_rate': 0.001, 'GPU': False, 'epoch': 100, 'epoch_num': 5, 'optimizer': 'Adam', 'model_architecture': 'ST', 'hidden_layer': 1, 'mean_list': [-2.9004206167, -2.3558172474, -1.5568911996452022, -1.2787787346000001], 'std_list': [0.43299614209176923, 0.5196112859274986, 1.2277285256477377, 1.0470026182568197], 'mad_list': [0.3657161829600001, 0.4066321777199999, 1.002357703865224, 0.88236819802], 'ratio_list': [1.1839676838668303, 1.2778410425878648, 1.2248407139621453, 1.1865824500545838]}
Epoch: 1 Loss=14.099
======== Iteration 0 ======

Epoch: 1 Loss=13.776
======== Iteration 0 ======
Evaluating model
[0.6333836348284573, 0.6381311739120041, 0.754743726506834, 0.5956199389873145]
[0.35115194, 0.29119205, 0.2396839, 0.40320778]
Training R2 score: 0.655
Training MSE score: 0.321
[-14.361097508854714, -1.1821029979087236, 0.577331759506615, -0.8480486083907486]
[0.47036478, 1.7219162, 0.45332816, 1.2292037]
Test R2 score: -3.953
Test MSE score: 0.969
Epoch: 1 Loss=4.341
======== Iteration 1 ======
Evaluating model
[0.8435149106773586, 0.7049138882127763, 0.8809869235885159, 0.7490259103135865]
[0.14988431, 0.2374527, 0.11630903, 0.25024652]
Training R2 score: 0.795
Training MSE score: 0.188
[-18.054512044515857, -1.1068413787836895, 0.6451421729115647, -0.4762553597313419]
[0.583459, 1.6625266, 0.38059887, 0.9819106]
Test R2 score: -4.748
Test MSE score: 0.902
[-35.89682008916943, -0.4885651924453316, 0.6391035439854527, -0.29718848460178604]
[1.1297998, 1.1746396, 0.38707554, 0.86280674]
Current score: -35.897
Best scor

Epoch: 1 Loss=15.504
======== Iteration 0 ======
Evaluating model
[0.6378571958785311, 0.15592367936464901, 0.8364704376725476, 0.7275406997596341]
[0.34686708, 0.67921937, 0.15981409, 0.27166945]
Training R2 score: 0.589
Training MSE score: 0.364
[-16.334310424808503, -0.24972921952719318, 0.592013641270841, -0.08196924067456224]
[0.53078556, 0.9861721, 0.43758127, 0.7196567]
Test R2 score: -4.018
Test MSE score: 0.669
Epoch: 1 Loss=3.376
======== Iteration 1 ======
Evaluating model
[0.16792381012670932, 0.8004956294344435, 0.9144873179012546, 0.8020479790444023]
[0.796978, 0.16053908, 0.08356979, 0.19737817]
Training R2 score: 0.671
Training MSE score: 0.310
[-9.067734897652974, 0.10197827320885311, 0.6598329966824819, -0.3242028786766289]
[0.30827928, 0.7086367, 0.36484239, 0.88077503]
Test R2 score: -2.158
Test MSE score: 0.566
[-4.391245935189564, 0.1388953418129728, 0.6495702719974572, -0.6472376379558196]
[0.16508275, 0.6795051, 0.37584952, 1.0956371]
Current score: -4.391
Best 

Epoch: 1 Loss=15.829
======== Iteration 0 ======
Evaluating model
[0.35655098393570117, 0.5698246853189746, 0.820183706196626, 0.6480538035360179]
[0.6163074, 0.34615755, 0.17573076, 0.35092592]
Training R2 score: 0.599
Training MSE score: 0.372
[-11.642682586923284, -0.42157169676620576, 0.5882534638637633, -0.21999172031704184]
[0.3871255, 1.1217744, 0.4416142, 0.81146044]
Test R2 score: -2.924
Test MSE score: 0.690
Epoch: 1 Loss=3.965
======== Iteration 1 ======
Evaluating model
[0.8598985928921082, 0.2571069606830053, 0.8996405714013271, 0.6239266938236943]
[0.13419174, 0.5977982, 0.098079205, 0.37498313]
Training R2 score: 0.660
Training MSE score: 0.301
[-7.5962609279514055, -0.4105215893396543, 0.6308072312848136, -0.07890086841177513]
[0.26322198, 1.1130548, 0.39597365, 0.71761584]
Test R2 score: -1.864
Test MSE score: 0.622
[-2.4345302886394076, 0.3028369001375515, 0.646529438517796, -0.1453512954136842]
[0.1051671, 0.5501374, 0.37911096, 0.76181436]
Current score: -2.435
Best

## Model training

In [ ]:
from visar.pytorch_regressor import pytorch_DNN_model
from visar.dataloader.pytorch_utils import compound_FP_loader
train_loader, test_loader, train_df, test_df, para_dict_DNN = compound_FP_loader(para_dict_DNN)


In [ ]:
pyDNN_model = pytorch_DNN_model(para_dict_DNN)
pyDNN_model.model_init()
pyDNN_model.model

In [ ]:
pyDNN_model.fit(train_loader, test_loader)

## generate viz files

In [ ]:
from visar.dataloader.pytorch_utils import compound_FP_loader
from visar.pytorch_regressor import pytorch_DNN_model

# prepare custom dataloader
custom_para_dict = {
    'task_list': ['activity'],   # a dummy column of float
    # input data related params:
    'dataset_file': './data/binding_mode_notation.csv',
    'feature_type': 'Morgan',
    'id_field': 'cid',   #
    'smiles_field': 'SMILES',  #
    'model_flag':'ST',
    'add_features': None,
    'frac_train': 1,
    'batch_size': 100,
    'normalize': False
}

custom_loader, custom_df, custom_para_dict = compound_FP_loader(custom_para_dict)

In [ ]:
# load previous model
import json
para_dict_DNN = json.load(open('./logs/VISAR_pytorch_demo/train_parameters.json','r'))

pyDNN_model = pytorch_DNN_model(para_dict_DNN)
pyDNN_model.model_init()
pyDNN_model.load_model()

In [ ]:
# load training data
train_loader, test_loader, train_df, test_df, para_dict_DNN = compound_FP_loader(para_dict_DNN,
                                                                                max_cutoff = 8000)

In [ ]:
pyDNN_model.para_dict['custom_id_field'] = custom_para_dict['id_field']
pyDNN_model.para_dict['custom_smiles_field'] = custom_para_dict['smiles_field']
pyDNN_model.para_dict['hidden_layer'] = 2
pyDNN_model.para_dict['model_architecture'] = 'RobustMT'
pyDNN_model.generate_viz_results(train_loader, train_df, 'KLIFS_global750_MT_rep3',
                           custom_loader = custom_loader, 
                           custom_df = custom_df)